In [84]:
import pandas as pd
import numpy as np
class CSymbol():                
    def __init__(self,specs={}):
                
        #get symbols specifications like swaps min volum max volum symbol name precision contract volum etc..
        self.specs=specs            
        #self.rates=pd.read_csv(specs["file_name"])    
        
        return None
    
#Not clear now    
class COperation():
    def __init__(self):
        return None

#    
class CPositionInfo():
    #self.lot=0.
    #self.type=0#or 1 0 shot 1 long
    #self.symbol class symbol
    #self.step_open #step open price
    #self.step_close #none or step to close price
    #self.current_profit profit in money
    
    def __init__(self):
        return None
    
#define custom strategy...
class CStrategy():    
    def __init__(self,name,simulator):
        self.name="Custom strategy name"
        self.simulator=simulator
        return None
    
    def run_strategy(self,model=None):
        while self.simulator.Turn():
            #do somthing indicator working then
            operation_list=[]
            simulator.Perform_OPS(Operations_list)            
        return None
    
#same as CStrategy but multiple strategy with shared simulator    
class CMultiStrategy():
    def __init__(self,strategy_list,simulator):
        return None

    
    
class CSimulator:    
    
    def __init__(self,enviroment,i_symbols=[],inital_step=0):        
        self.Symbols=[] 
        self.PosInfo=[]#list of active positions
        self.PosHist=[]#history all positions
        self.enviroment=enviroment
        self.initial_balance=enviroment["balance"]
        self.current_balance=self.initial_balance
        self.current_equity=self.initial_balance
        for x in i_symbols:
            newsym=CSymbol(x)
            self.Symbols.append(newsym)
            
        
        self.syncronize_symbols()#syncronize all symbols indexs
        self.set_cursor(inital_step)#init cursos at step 0
        return None
    
    #TODO syncronize all symbols datetimes
    def syncronize_symbols(self):
        return None
    
    #if we whant interactive session we set or initialize cursor at step we needed
    def set_cursor(self,step=0):
        self.current_step=step
        
        return None
    
    #for interactive session return true while step < len(rates)
    #example usage while simulator.Turn():
    #                    perform_ops...    
    def Turn(self):
        
        self.calcSates()
        return False
    
    #use together with Turn
    def Perform_OPS(operation):
        return None
    #return n pandas frames with all Symbols sycronizeds
    def getRatesHist(self):
        return None
    
    #return current stadisitical opbjects
    def getStats(self):
        return None
    
    #return all current stadistical object from step 0 to current state
    def getStatsHistory(self):
        return None
    
    #private method called at end of turn
    def calcSates(self):
        return None
    
    #return all active positions
    def PositionsCount():
        return None
    
class CCustomIndicator():
    def __init__(self):
        return None
    
def main():
    all_symbols=[]
    specs={"file_name":'data/EURUSD.csv',
           "symbol_name":"EURUSD",
           "precision":5,
           "swap_long":-0.8,
           "swap_short":-0.4,
           "contract_volum":100000}
    all_symbols.append(specs)
    
    specs={"file_name":'data/GBPUSD.csv',
           "symbol_name":"GBPUSD",
           "precision":5,
           "swap_long":-0.8,
           "swap_short":-0.4,
           "contract_volum":100000}
    all_symbols.append(specs)
    initial_step=0
    enviroment={"balance":10000}
    simulator=CSimulator(enviroment,all_symbols,initial_step)
    
    
    #usage method 1 
    #once simulator initialited and symbols sycrnonized we can use them on our ML/DL model
    #we can uset getRatesHist to perform our models
    #then we can define a custom class stragey with
    
    strategy=CStrategy("strategyname",simulator)
    x_train=simulator.getRatesHist()
    #train you model operations
    model=None
    strategy.run_strategy(model)
    
    #or we can use on Q-learning or NEAT or genetic algorithm
    strategy=CStrategy("strategyname",simulator)    
    strategy.run_strategy()

In [85]:
main()